In [ ]:
from tensorflow.keras.datasets import imdb

(X_train,y_train),(X_test,y_test) = imdb.load_data(num_words=10000)

word_index = imdb.get_word_index()

d = dict([(value,key) for (key,value) in word_index.items()])

" ".join([d.get(i-3,"?") for i in X_train[6]])

In [ ]:
import pandas as pd
import numpy as np
from itertools import combinations

def CreateCalculatedField(df, primary_key, calc_instructions, include_all=1):

    '''
    calc_instructions = [
    {'type': 'sum', 'value1': 'LENDING', 'name': 'TOTAL_LENDING'},
    {'type': 'weighted_average', 'value1': 'LENDING', 'value2': 'INTEREST_RATE', 'name': 'WEIGHTED_INTEREST'},
    {'type': 'ratio', 'value1': 'RENEWED_AMOUNT', 'value2': 'MATURED_AMOUNT', 'name': 'RENEWAL_RATE'}
    ]

    output = CreateCalculatedField(final_df, ['BRANCHNAME', 'CITY', 'LOB', 'DURATION'], calc_instructions)


    '''

    base_aggs = {}
    
    # Collect all fields we need
    for calc in calc_instructions:
        if calc['type'] == 'sum':
            base_aggs[calc['name']] = (calc['value1'], 'sum')
        elif calc['type'] == 'weighted_average':
            base_aggs[f"__{calc['name']}_NUM"] = (
                calc['value2'], lambda x, col=calc['value1']: (df.loc[x.index, col] * x).sum()
            )
            base_aggs[f"__{calc['name']}_DEN"] = (calc['value1'], 'sum')
        elif calc['type'] == 'ratio':
            base_aggs[f"__{calc['name']}_NUM"] = (calc['value1'], 'sum')
            base_aggs[f"__{calc['name']}_DEN"] = (calc['value2'], 'sum')

    # Base groupby
    grouped = df.groupby(primary_key, dropna=False).agg(**base_aggs).reset_index()

    # Compute post-aggregates
    for calc in calc_instructions:
        if calc['type'] == 'sum':
            continue
        elif calc['type'] == 'weighted_average':
            num = grouped[f"__{calc['name']}_NUM"]
            den = grouped[f"__{calc['name']}_DEN"]
            grouped[calc['name']] = np.where(den != 0, num / den, np.nan)
            grouped.drop(columns=[f"__{calc['name']}_NUM", f"__{calc['name']}_DEN"], inplace=True)
        elif calc['type'] == 'ratio':
            num = grouped[f"__{calc['name']}_NUM"]
            den = grouped[f"__{calc['name']}_DEN"]
            grouped[calc['name']] = np.where(den != 0, num / den, np.nan)
            grouped.drop(columns=[f"__{calc['name']}_NUM", f"__{calc['name']}_DEN"], inplace=True)

    result_frames = [grouped.copy()]

    # Rollup combinations
    if include_all:
        for r in range(1, len(primary_key)):
            for group_cols in combinations(primary_key, r):
                temp = df.copy()
                for col in primary_key:
                    if col not in group_cols:
                        temp[col] = 'All'
                temp_group = CreateCalculatedField(temp, primary_key, calc_instructions, include_all=0)
                result_frames.append(temp_group)

        # Full 'All' row
        temp = df.copy()
        for col in primary_key:
            temp[col] = 'All'
        temp_group = CreateCalculatedField(temp, primary_key, calc_instructions, include_all=0)
        result_frames.append(temp_group)

    return pd.concat(result_frames, ignore_index=True)



calc_instructions = [
    {'type': 'sum', 'value1': 'LENDING', 'name': 'TOTAL_LENDING'},
    {'type': 'weighted_average', 'value1': 'LENDING', 'value2': 'INTEREST_RATE', 'name': 'WEIGHTED_INTEREST'},
    {'type': 'ratio', 'value1': 'RENEWED_AMOUNT', 'value2': 'MATURED_AMOUNT', 'name': 'RENEWAL_RATE'}
]

output = CreateCalculatedField(final_df, ['BRANCHNAME', 'CITY', 'LOB', 'DURATION'], calc_instructions)

output
\
from itertools import product,permutations,combinations


branches = ['Fenway Park','Wrigely Field','The Forum','Maple Leaf Gardens','GM Place','Safeco','Lambeau Stadium']
city = ['Boston','Chicago','Philly','Toronto','Vancouver','Chicago','Seattle','Green Bay']
lob = ['Retail','Corporate','Mid Market']
duration = ['1) Less than 30d','2) 30D - 1Y', '3) 1Y - 5Y','4)5Y+']
list_of_lists = [branches,city,lob,duration]

In [ ]:
from DataSets import GenerateFakeMemberDF



df = GenerateFakeMemberDF(100,20)


branches = ['Fenway Park','Wrigely Field','The Forum','Maple Leaf Gardens','GM Place','Safeco','Lambeau Stadium']
city = ['Boston','Chicago','Philly','Toronto','Vancouver','Chicago','Seattle','Green Bay']
lob = ['Retail','Corporate','Mid Market']
duration = ['1) Less than 30d','2) 30D - 1Y', '3) 1Y - 5Y','4)5Y+']

list_of_lists = [branches,city,lob,duration]

df1 = pd.DataFrame(df['MEMBERNBR'].unique(),columns=['MEMBERNBR'])
df1['BRANCHNAME'] = [np.random.choice(branches) for x in range(len(df1))]
df1['CITY'] = [np.random.choice(city) for x in range(len(df1))]
df1['LOB'] = [np.random.choice(lob) for x in range(len(df1))]
df1['DURATION'] = [np.random.choice(duration) for x in range(len(df1))]

final_df = df.merge(df1,on='MEMBERNBR',how='left')
final_df['INTEREST_RATE'] = final_df['MONTH'].apply(lambda x:np.random.uniform(0.02, 0.07))
final_df['MATURED_AMOUNT'] = final_df['LENDING'].apply(lambda x:x*np.random.uniform(0, 1)) 
final_df['RENEWED_AMOUNT'] = final_df['MATURED_AMOUNT'].apply(lambda x:x*.95) 

In [ ]:
import inspect
import difflib

def compare_functions(func1, func2):
    f1_lines = inspect.getsource(func1).splitlines()
    f2_lines = inspect.getsource(func2).splitlines()
    diff = difflib.unified_diff(f1_lines, f2_lines, lineterm='')
    return '\n'.join(diff)

compare_functions(ColumnStatisticalReview,ColumnStatisticalReview1)

In [ ]:
from DataSets import GenerateFakeMemberDF
from FeatureEngineering import CreateRandomDFColumn
df = GenerateFakeMemberDF(10000,12)
CreateRandomDFColumn(df,['Fenway','Yankee Stadium','Wrigly'],'Stadium')
CreateRandomDFColumn(df,['John','Mark','Harry','Sally'],'Vendor')

In [2]:
import re
import pandas as pd

df = pd.DataFrame({
    'text': [
        'Transaction from BCCA001 and ONCA-fund',
        'Nothing to remove here',
        'Another bcca999 item',
        'Words withonca inside'
    ]
})

# Regex to remove entire words containing 'bcca' or 'onca' (case-insensitive)
pattern = r'\b\w*(bcca|onca)\w*\b'

# Clean the text
df['cleaned'] = df['text'].str.replace(pattern, '', regex=True, case=False).str.replace(r'\s+', ' ', regex=True).str.strip()

print(df)

                                     text                     cleaned
0  Transaction from BCCA001 and ONCA-fund  Transaction from and -fund
1                  Nothing to remove here      Nothing to remove here
2                    Another bcca999 item                Another item
3                   Words withonca inside                Words inside


In [ ]:
from sklearn.metrics import precision_recall_curve

precisions, recalls, thresholds = precision_recall_curve(y_train_5, y_scores)


plt.plot(thresholds, precisions[:-1], "b--", label="Precision", linewidth=2)
plt.plot(thresholds, recalls[:-1], "g-", label="Recall", linewidth=2)
plt.vlines(threshold, 0, 1.0, "k", "dotted", label="threshold")
[...]  # beautify the figure: add grid, legend, axis, labels, and circles
plt.show()


plt.plot(recalls, precisions, linewidth=2, label="Precision/Recall curve")
[...]  # beautify the figure: add labels, grid, legend, arrow, and text
plt.show()

from sklearn.metrics import ConfusionMatrixDisplay

y_train_pred = cross_val_predict(sgd_clf, X_train_scaled, y_train, cv=3)
ConfusionMatrixDisplay.from_predictions(y_train, y_train_pred)
plt.show()

ConfusionMatrixDisplay.from_predictions(y_train, y_train_pred,
                                        normalize="true", values_format=".0%")
plt.show()

sample_weight = (y_train_pred != y_train)
ConfusionMatrixDisplay.from_predictions(y_train, y_train_pred,
                                        sample_weight=sample_weight,
                                        normalize="true", values_format=".0%")
plt.show()

In [ ]:
import matplotlib.pyplot as plt

def plot_digit(image_data):
    image = image_data.reshape(28, 28)
    plt.imshow(image, cmap="binary")
    plt.axis("off")

some_digit = X[0]
plot_digit(some_digit)
plt.show()

In [ ]:
metrics = ['DEPOSIT','LENDING','TXN_COUNT','TXN_VALUE','RECORD_COUNT']
index=['BRANCHNAME','CITY','LOB','DURATION']

column='MONTH'

final_df['RECORD_COUNT']=1

def CreateMultiplePivotTableFromTimeSeries(df,index_list,metric_list,column):
    '''
    Function to utilize when Attempting to Create Multip[le Times Series. Specifically Multiple Metrics, and Multiple Index's
    



    '''
    
    
    final_df = pd.DataFrame()
    
    # Iterate through all Possible Metrics Selected.
    for metric in metric_list:
        all_df = CreatePivotTableFromTimeSeries(df=df,index=None,columns=column,values=metric,aggfunc='sum') 
        cols = list(all_df.columns)
        all_df = all_df.reset_index(drop=True)
        all_df['METRIC'] = metric
        cols.insert(0,'METRIC')

        for key in index:
            cols.insert(0,key)
            all_df[key] = 'All'

        final_df = pd.concat([final_df,all_df[cols]])

        # Iterate through all Index Items Individually
        for key in index_list:
            temp = CreatePivotTableFromTimeSeries(df,index=key,
                                                  values=metric,
                                                  columns=column).reset_index() 
            for missing in [x for x in index if x != key]:
                temp[missing] = 'All'
            temp['METRIC'] = metric
            final_df = pd.concat([final_df,temp])
        
        # Add Value for Metric with Entire Index Combination
        temp = CreatePivotTableFromTimeSeries(df,index=index_list,values=metric,columns=column).reset_index()
        temp['METRIC'] = metric
        final_df = pd.concat([final_df,temp])
    
    return final_df

e = CreateMultiplePivotTableFromTimeSeries(df=final_df[final_df['MONTH']<3],
                                       index_list= index,
                                       metric_list=metrics,
                                       column= 'MONTH')


def CreatePivotTableFromTimeSeries(df,
                                   index,
                                   columns,
                                   values,
                                   aggfunc='sum',
                                   skipna=True):
    
    '''
    Function to Summaryize a Time Series Dataframe into a Pivot. Creating a number of critical Metrics.
    
    
    
    '''
    
    # 1. Pivot
    if index==None:
        df1 = df.pivot_table(columns=columns,values=values,aggfunc=aggfunc)
    else:
        df1 = df.pivot_table(index=index, columns=columns, values=values, aggfunc=aggfunc)

    # 2. Capture original month columns IMMEDIATELY after pivot
    month_cols = df1.columns.tolist()
 
    # 3. Add rolling window stats
    if len(month_cols) >= 3:
        df1['AVG_3M'] = df1[month_cols[-3:]].mean(axis=1, skipna=skipna)
        df1['CHG_3M'] = df1[month_cols[-1]]-df1[month_cols[-3]]
        try:
            df1['PERC_CHG_3M'] = df1['CHG_3M']/df1[month_cols[-3]]
        except:
            df1['PERC_CHG_3M'] = 0
    
    if len(month_cols) >= 6:
        df1['AVG_6M'] = df1[month_cols[-6:]].mean(axis=1, skipna=skipna)
        df1['CHG_6M'] = df1[month_cols[-1]]-df1[month_cols[-6]]
        try:
            df1['PERC_CHG_6M'] = df1['CHG_6M']/df1[month_cols[-6]]
        except:
            df1['PERC_CHG_6M'] = 0
            
    if len(month_cols) >= 12:
        df1['AVG_12M'] = df1[month_cols[-12:]].mean(axis=1, skipna=skipna)
        df1['CHG_12M'] = df1[month_cols[-1]]-df1[month_cols[-12]]
        try:
            df1['PERC_CHG_12M'] = df1['CHG_12M']/df1[month_cols[-12]]
        except:
            df1['PERC_CHG_12M'] = 0

    df1['CHG_DF']  = df1[month_cols[-1]]-df1[month_cols[0]]
    df1['AVG_DF'] = df1[month_cols[-1:]].mean(axis=1, skipna=skipna)
    df1['PERC_CHG_DF'] = df1['AVG_DF']/df1[month_cols[-1]]

    
    # 4. Now calculate global stats **only using the original month columns**
    stats = pd.DataFrame({
        'MEAN': df1[month_cols].mean(axis=1, skipna=skipna),
        'STD': df1[month_cols].std(axis=1, skipna=skipna),
        'MAX': df1[month_cols].max(axis=1, skipna=skipna),
        'MIN': df1[month_cols].min(axis=1, skipna=skipna),
        'COUNT': df1[month_cols].count(axis=1)
    })

    # 5. Merge the stats
    df1 = pd.concat([df1, stats], axis=1)
    
    return df1.fillna(0)